In [ ]:
import random
from rail_sim.models.passenger import Passenger
from rail_sim.models.line import Line
from rail_sim.models.train import Train
from rail_sim.network.network import Network

# Define a simple line and network
line = Line(
    name="T1",
    stops=["Central", "Town Hall", "Wynyard", "Circular Quay"],
    travel_times=[2.0, 3.0, 4.0],
    fleet_size=3,  # Updated fleet size to be greater than 2
)
net = Network(transfer_time=2.0)
net.add_line(line)

# Create a few passengers with forward and backward trips
rng = random.Random(42)
passengers = {}
pid = 1
for _ in range(8):
    oi = rng.randrange(0, len(line.stops))
    di = rng.randrange(0, len(line.stops))
    if oi == di:
        continue  # Ensure origin and destination are different
    p = Passenger(id=pid, origin_id=line.stops[oi], dest_id=line.stops[di], created_at=0.0, queued_at=0.0)
    passengers[pid] = p
    net.stations[p.origin_id].add_passenger_to_queue(p.id, line.name)
    pid += 1

# Create multiple trains for the line
trains = [Train(train_id=f"T1-{i:03}", line=line.name, capacity=4) for i in range(3)]

# Start simulation for each train
for train in trains:
    current_time = 0.0
    station_idx = 0
    train.current_station = line.stops[station_idx]
    print(f"Start simulation for {train.id}")
    while True:
        station_name = line.stops[station_idx]
        station = net.get_station(station_name)
        print(f"\nTime {current_time:.1f} min | Train at {station_name}")

        # 1) Alight
        alighting = train.alight_passengers(station_name, passengers)
        for pid in alighting:
            passengers[pid].alighted_at = current_time
        if alighting:
            print(f"  Alighted: {alighting}")

        # 2) Board
        to_board_ids = station.pop_for_boarding(line.name, train.available_capacity)
        boarded_ids = train.board_passengers(to_board_ids)
        for pid in boarded_ids:
            passengers[pid].boarded_at = current_time
        if boarded_ids:
            print(f"  Boarded:  {boarded_ids} (occ={train.occupancy}/{train.capacity})")
        else:
            print(f"  No boarding (occ={train.occupancy}/{train.capacity})")

        # 3) If at last station, end; else move to next station
        if station_idx == len(line.stops) - 1:
            print("Reached terminal. Ending.")
            break

        # Move to next stop
        next_idx = station_idx + 1
        travel_time = line.travel_times[station_idx]
        current_time += travel_time
        station_idx = next_idx

# Report simple metrics
served = [p for p in passengers.values() if p.alighted_at is not None]
left_onboard = train.onboard[:]  # any not alighted by terminal (should be none if all dests <= terminal)
waiting_remaining = sum(st.get_queue_length(line.name) for st in net.stations.values())

print(f"\nSummary:")
print(f"  Passengers created: {len(passengers)}")
print(f"  Passengers served:  {len(served)}")
print(f"  Left onboard:       {left_onboard}")
print(f"  Still waiting:      {waiting_remaining}")

for p in sorted(served, key=lambda x: x.id):
    print(f"    P{p.id}: wait={p.wait_time:.1f} min, in-vehicle={p.in_vehicle_time:.1f} min, total={p.total_journey_time:.1f} min")